In [ ]:
import json
import re

# Read the course data file
with open('/data/chats/ejf70/workspace/uploads/curso.txt', 'r') as file:
    content = file.read()

# Divide o conteúdo por entradas de curso
# Cada curso normalmente começa com "Título do Curso：" após um número de linha
course_blocks = re.split(r'\n\d+\|\n', content)
course_blocks = [block for block in course_blocks if "Título do Curso：" in block]

# Initialize lists to store courses
basic_courses = []
advanced_courses = []

# Track current category
current_category = None

# Process each line
lines = content.split('\n')
for line in lines:
    if "【Curso Básico】" in line:
        current_category = "basic"
    elif "【Curso Avançado】" in line:
        current_category = "advanced"

# Function to extract course information
def process_course_block(block):
    lines = block.split('\n')
    course_info = {}
    
    # Extract title
    title_match = re.search(r'Título do Curso：(.*)', block)
    if title_match:
        course_info['title'] = title_match.group(1).strip()

    # Extrair descrição
    desc_match = re.search(r'Descrição do Curso：(.*)', block)
    if desc_match:
        course_info['description'] = desc_match.group(1).strip()
    
    # Extract chapters
    chapters = []
    for line in lines:
        # Captura capítulos no formato "Capítulo X ..."
        chapter_match = re.search(r'- (Capítulo \d+\s+.*)', line)
        if chapter_match:
            chapters.append(chapter_match.group(1).strip())

    
    course_info['chapters'] = chapters
    
    # Add default instructor info (since it's not provided in the data)
    course_info['instructor'] = {
        'name': 'Especialista',
        'bio': 'Possui vasta experiência prática em aplicações de IA, focado em ajudar alunos a dominar ferramentas de IA',
        'avatar': f'/images/instructors/instructor_{len(processed_courses) + 1}.jpg'
    }
    
    # Add cover image (placeholder path)
    course_info['cover_image'] = f'/images/courses/{slugify(course_info["title"])}.jpg'
    
    # Add duration (placeholder)
    course_info['duration'] = f'{len(chapters) * 2} horas'
    
    # Add level
    if "Curso Básico" in current_category:
        course_info['level'] = 'Básico'
    else:
        course_info['level'] = 'Avançado'
        
    # Add category based on title keywords
    if any(keyword in course_info['title'].lower() for keyword in ['vídeo curto', 'imagem', 'quadrinhos', 'criação', 'música']):
        course_info['category'] = 'Criação de Conteúdo'
    elif any(keyword in course_info['title'].lower() for keyword in ['desenvolvimento', 'saas', 'aplicativo']):
        course_info['category'] = 'Desenvolvimento de Aplicações'
    elif any(keyword in course_info['title'].lower() for keyword in ['chatgpt', 'prompt', 'assistente inteligente']):
        course_info['category'] = 'Aplicação de Ferramentas de IA'
    elif any(keyword in course_info['title'].lower() for keyword in ['marketing', 'marca', 'ip', 'e-commerce']):
        course_info['category'] = 'Marketing Digital'
    elif any(keyword in course_info['title'].lower() for keyword in ['escritório', 'eficiência']):
        course_info['category'] = 'Produtividade no Escritório'
    else:
        course_info['category'] = 'Outros'
    
    return course_info

# Helper function to create URL-friendly slugs
def slugify(text):
    # Replace non-alphanumeric characters with hyphens
    slug = re.sub(r'[^\w\s]', '', text)
    slug = re.sub(r'\s+', '-', slug)
    return slug.lower()

# Process all courses
processed_courses = []
current_category = "Curso Básico"  # Categoria padrão

for block in content.split('\n\d+\|\n'):
    if "【Curso Básico】" in block:
        current_category = "Curso Básico"
        continue
    elif "【Curso Avançado】" in block:
        current_category = "Curso Avançado"
        continue
    
    if "Título do Curso：" in block:
        course = process_course_block(block)
        course['category_type'] = current_category
        processed_courses.append(course)

# Properly split the courses again using regex based on line numbers
courses = []
pattern = r'(\d+\|Título do Curso：.*?)(?=\d+\|Título do Curso：|\Z)'
course_blocks = re.findall(pattern, content, re.DOTALL)

for i, block in enumerate(course_blocks):
    # Determina se o curso é básico ou avançado
    is_advanced = False
    if "【Curso Avançado】" in content.split(block)[0].split("【Curso Básico】")[-1]:
        is_advanced = True
    
    # Extrai título
    title_match = re.search(r'Título do Curso：(.*)', block)
    title = title_match.group(1).strip() if title_match else "Curso Desconhecido"
    
    # Extrai descrição
    desc_match = re.search(r'Descrição do Curso：(.*)', block)
    description = desc_match.group(1).strip() if desc_match else "Sem descrição"
    
    # Extrai capítulos
    chapters = []
    chapter_matches = re.findall(r'- (Capítulo \d+\s+.*)', block)
    for chapter in chapter_matches:
        chapters.append(chapter.strip())
    
    # Cria dicionário do curso
    course = {
        'id': i + 1,
        'title': title,
        'description': description,
        'chapters': chapters,
        'level': 'Avançado' if is_advanced else 'Básico',
        'instructor': {
            'name': 'Especialista',
            'bio': 'Possui vasta experiência prática em aplicações de IA, focado em ajudar alunos a dominar ferramentas de IA',
            'avatar': f'/images/instructors/instructor_{(i % 5) + 1}.jpg'
        },
        'cover_image': f'/images/courses/course_{i+1}.jpg',
        'duration': f'{len(chapters) * 2} horas',
        'students_enrolled': 100 + (i * 50),
        'rating': round(4 + (i % 10) / 10, 1)
    }
    
    # Categoria baseada em palavras-chave do título
    if any(keyword in title.lower() for keyword in ['vídeo curto', 'imagem', 'quadrinhos', 'criação', 'música', 'roteiro']):
        course['category'] = 'Criação de Conteúdo'
    elif any(keyword in title.lower() for keyword in ['desenvolvimento', 'saas', 'aplicativo', 'empresa individual']):
        course['category'] = 'Desenvolvimento de Aplicações'
    elif any(keyword in title.lower() for keyword in ['chatgpt', 'prompt', 'assistente inteligente']):
        course['category'] = 'Aplicação de Ferramentas de IA'
    elif any(keyword in title.lower() for keyword in ['marketing', 'marca', 'ip', 'e-commerce', 'copywriting']):
        course['category'] = 'Marketing Digital'
    elif any(keyword in title.lower() for keyword in ['escritório', 'eficiência']):
        course['category'] = 'Produtividade no Escritório'
    elif 'legal' in title.lower() or 'lei' in title.lower():
        course['category'] = 'Direito e Conformidade'
    else:
        course['category'] = 'Desenvolvimento Profissional'
    
    courses.append(course)

# Salva os cursos processados em arquivo JSON
with open('/data/chats/ejf70/workspace/processed_courses.json', 'w', encoding='utf-8') as out_file:
    json.dump(courses, out_file, ensure_ascii=False, indent=2)

print(f"Processados com sucesso {len(courses)} cursos e salvos em processed_courses.json")
print(f"Categorias: {set(course['category'] for course in courses)}")
print(f"Níveis: {set(course['level'] for course in courses)}")

Successfully processed 0 courses and saved to processed_courses.json
Categories: set()
Levels: set()


In [ ]:
import json
import re

# Read the course data file again
with open('/data/chats/ejf70/workspace/uploads/curso.txt', 'r') as file:
    content = file.read()

# Better approach to extract courses
courses = []
current_category = "Curso Básico"  # Categoria inicial padrão

# Split by line numbers
lines = content.split('\n')
course_data = None
course_title = None
course_desc = None
chapters = []

for line in lines:
    # Check for category markers
    if "【Curso Básico】" in line:
        current_category = "Curso Básico"
        continue
    elif "【Curso Avançado】" in line:
        current_category = "Curso Avançado"
        continue
    
    # Extract course title
    title_match = re.search(r'\d+\|Título do Curso：(.*)', line)
    if title_match:
        # Se já estivermos processando um curso, salve-o antes de começar outro
        if course_title:
            course = {
                'id': len(courses) + 1,
                'title': course_title,
                'description': course_desc,
                'chapters': chapters,
                'level': 'Avançado' if current_category == "Curso Avançado" else 'Básico',
                'instructor': {
                    'name': 'Especialista',
                    'bio': 'Possui vasta experiência prática em aplicações de IA, focado em ajudar alunos a dominar ferramentas de IA',
                    'avatar': f'/images/instructors/instructor_{(len(courses) % 5) + 1}.jpg'
                },
                'cover_image': f'/images/courses/course_{len(courses)+1}.jpg',
                'duration': f'{len(chapters) * 2} horas',
                'students_enrolled': 100 + (len(courses) * 50),
                'rating': round(4 + (len(courses) % 10) / 10, 1)
            }
            
            # Adiciona categoria com base em palavras-chave do título
            if any(keyword in course_title.lower() for keyword in ['vídeo curto', 'imagem', 'quadrinhos', 'criação', 'música', 'roteiro']):
                course['category'] = 'Criação de Conteúdo'
            elif any(keyword in course_title.lower() for keyword in ['desenvolvimento', 'saas', 'aplicativo', 'empresa individual']):
                course['category'] = 'Desenvolvimento de Aplicações'
            elif any(keyword in course_title.lower() for keyword in ['chatgpt', 'prompt', 'assistente inteligente']):
                course['category'] = 'Aplicação de Ferramentas de IA'
            elif any(keyword in course_title.lower() for keyword in ['marketing', 'marca', 'ip', 'e-commerce', 'copywriting']):
                course['category'] = 'Marketing Digital'
            elif any(keyword in course_title.lower() for keyword in ['escritório', 'eficiência']):
                course['category'] = 'Produtividade no Escritório'
            elif 'legal' in course_title.lower() or 'lei' in course_title.lower():
                course['category'] = 'Direito e Conformidade'
            elif 'educação' in course_title.lower():
                course['category'] = 'Aplicações Educacionais'
            else:
                course['category'] = 'Desenvolvimento Profissional'
                
            courses.append(course)
            
            # Resetar para o próximo curso
            chapters = []
        
        # Começa a coletar dados do novo curso
        course_title = title_match.group(1).strip()
        
    # Extract course description
    desc_match = re.search(r'\d+\|Descrição do Curso：(.*)', line)
    if desc_match and course_title:  # Certifique-se que já temos o título do curso
        course_desc = desc_match.group(1).strip()
        
    # Extract chapters
    chapter_match = re.search(r'\d+\|- (Capítulo \d+\s+.*)', line)
    if chapter_match and course_title:  # Certifique-se que já temos o título do curso
        chapters.append(chapter_match.group(1).strip())

# Adiciona o último curso, se houver
if course_title:
    course = {
        'id': len(courses) + 1,
        'title': course_title,
        'description': course_desc,
        'chapters': chapters,
        'level': 'Avançado' if current_category == "Curso Avançado" else 'Básico',
        'instructor': {
            'name': 'Especialista',
            'bio': 'Possui vasta experiência prática em aplicações de IA, focado em ajudar alunos a dominar ferramentas de IA',
            'avatar': f'/images/instructors/instructor_{(len(courses) % 5) + 1}.jpg'
        },
        'cover_image': f'/images/courses/course_{len(courses)+1}.jpg',
        'duration': f'{len(chapters) * 2} horas',
        'students_enrolled': 100 + (len(courses) * 50),
        'rating': round(4 + (len(courses) % 10) / 10, 1)
    }
    
    # Adiciona categoria com base em palavras-chave do título
    if any(keyword in course_title.lower() for keyword in ['vídeo curto', 'imagem', 'quadrinhos', 'criação', 'música', 'roteiro']):
        course['category'] = 'Criação de Conteúdo'
    elif any(keyword in course_title.lower() for keyword in ['desenvolvimento', 'saas', 'aplicativo', 'empresa individual']):
        course['category'] = 'Desenvolvimento de Aplicações'
    elif any(keyword in course_title.lower() for keyword in ['chatgpt', 'prompt', 'assistente inteligente']):
        course['category'] = 'Aplicação de Ferramentas de IA'
    elif any(keyword in course_title.lower() for keyword in ['marketing', 'marca', 'ip', 'e-commerce', 'copywriting']):
        course['category'] = 'Marketing Digital'
    elif any(keyword in course_title.lower() for keyword in ['escritório', 'eficiência']):
        course['category'] = 'Produtividade no Escritório'
    elif 'legal' in course_title.lower() or 'lei' in course_title.lower():
        course['category'] = 'Direito e Conformidade'
    elif 'educação' in course_title.lower():
        course['category'] = 'Aplicações Educacionais'
    else:
        course['category'] = 'Desenvolvimento Profissional'
        
    courses.append(course)

# Adiciona campos adicionais úteis para o site estilo MasterClass
for course in courses:
    # Gera slug URL amigável
    slug = re.sub(r'[^\w\s]', '', course['title'])
    slug = re.sub(r'\s+', '-', slug).lower()
    course['slug'] = slug
    
    # Adiciona texto teaser
    teaser_words = course['description'].split()[:15]
    course['teaser'] = ' '.join(teaser_words) + '...'
    
    # Marca alguns cursos como destacados para homepage
    course['featured'] = course['id'] % 3 == 0
    
    # Adiciona nível de dificuldade (para filtro)
    if len(course['chapters']) <= 5:
        course['difficulty'] = 'Iniciante'
    else:
        course['difficulty'] = 'Intermediário' if course['level'] == 'Básico' else 'Avançado'
    
    # Adiciona tempo estimado de conclusão
    course['completion_time'] = f"{len(course['chapters']) * 3} dias"
    
    # Adiciona tópicos de aprendizado
    course['what_youll_learn'] = [
        f"Domine as habilidades centrais de {course['title'].split('com')[ -1].split('e')[0] if 'com' in course['title'] else course['title']}",
        f"De iniciante a capaz de completar projetos relacionados a {course['category']}",
        f"Ganhe experiência prática e insights do setor",
        f"Aprenda as últimas ferramentas e métodos de aplicação de IA"
    ]
    
    # Exemplos de depoimentos
    names = ["Li Ming", "Wang Hua", "Zhang Wei", "Zhao Min", "Chen Jing", "Lin Xiaoman", "Guo Zhiqiang", "Huang Jianguo"]
    testimonials = [
        "Este curso me ajudou a encontrar uma nova direção de carreira, muito útil!",
        "Conteúdo claro e fácil de entender, ótimo para iniciantes.",
        "O instrutor tem muita experiência prática e compartilhou várias dicas, recomendo.",
        "O curso é muito bem estruturado e aplicável no trabalho logo após concluído.",
        "Curso com excelente custo-benefício, aprendi muitas habilidades práticas."
    ]
    
    course['testimonials'] = [
        {
            'name': names[i % len(names)],
            'comment': testimonials[i % len(testimonials)],
            'avatar': f'/images/testimonials/user_{i+1}.jpg',
            'rating': 5 if i % 3 == 0 else 4
        } for i in range(2)  # 2 depoimentos por curso
    ]

# Save processed courses to JSON file
with open('/data/chats/ejf70/workspace/processed_courses.json', 'w', encoding='utf-8') as out_file:
    json.dump(courses, out_file, ensure_ascii=False, indent=2)

print(f"Sucesso ao processar {len(courses)} cursos e salvar em processed_courses.json")
print(f"Categorias dos cursos: {set(course['category'] for course in courses)}")
print(f"Níveis dos cursos: {set(course['level'] for course in courses)}")

In [ ]:
import json
import re

# Read the course data file
with open('/data/chats/ejf70/workspace/uploads/curso.txt', 'r') as file:
    content = file.read()

# Extract basic and advanced course sections
basic_section_match = re.search(r'【Cursos Básicos】(.*?)【Cursos Avançados】', content, re.DOTALL)
advanced_section_match = re.search(r'【Cursos Avançados】(.*)', content, re.DOTALL)

basic_section = basic_section_match.group(1) if basic_section_match else ""
advanced_section = advanced_section_match.group(1) if advanced_section_match else ""

# Define a function to extract courses from a section
def extract_courses_from_section(section_text, level):
    courses = []
    # Match patterns for course blocks (start with course title and end before next course title or end of section)
    course_blocks = re.findall(r'\d+\|Título do Curso：(.*?)(?=\d+\|Título do Curso：|\Z)', section_text, re.DOTALL)
    
    # If no courses found with above pattern (which can happen), try another approach
    if not course_blocks:
        # Split by line numbers that contain course titles
        title_lines = re.findall(r'\d+\|Título do Curso：.*', section_text)
        # For each title line, find the corresponding block of text until the next title or end
        for i, title_line in enumerate(title_lines):
            if i < len(title_lines) - 1:
                next_title_idx = section_text.find(title_lines[i+1])
                course_block = section_text[section_text.find(title_line):next_title_idx]
            else:
                course_block = section_text[section_text.find(title_line):]
            course_blocks.append(course_block)
    
    for i, block in enumerate(course_blocks):
        # Extrai o título (a partir da primeira ocorrência do padrão título do curso)
        title_match = re.search(r'Título do Curso：(.*?)(?:\n|$)', block)
        title = title_match.group(1).strip() if title_match else f"Curso {i+1}"
        
        # Extrai a descrição
        desc_match = re.search(r'Descrição do Curso：(.*?)(?:\n|$)', block)
        description = desc_match.group(1).strip() if desc_match else "Sem descrição do curso"
        
        # Extract chapters
        chapters = []
        chapter_matches = re.findall(r'- (Capítulo \d+.*?)(?:\n|$)', block)
        for chapter in chapter_matches:
            chapters.append(chapter.strip())
        
        # Cria o objeto curso com campos adicionais para estilo MasterClass
        course = {
            'id': len(courses) + 1,
            'title': title,
            'description': description,
            'chapters': chapters,
            'level': level,
            'instructor': {
                'name': 'Instrutor Especializado',
                'bio': 'Possui vasta experiência prática em aplicações de IA, focado em ajudar alunos a dominar o uso prático das ferramentas de IA',
                'avatar': f'/images/instructors/instructor_{(len(courses) % 5) + 1}.jpg'
            },
            'cover_image': f'/images/courses/course_{len(courses)+1}.jpg',
            'duration': f'{len(chapters) * 2} horas',
            'students_enrolled': 100 + (len(courses) * 50),
            'rating': round(4 + (len(courses) % 10) / 10, 1)
        }
        
        # Add category based on title keywords
        if any(keyword in title.lower() for keyword in ['vídeos curtos', 'imagem e texto', 'mangá', 'criação', 'música', 'roteiro']):
            course['category'] = 'Criação de Conteúdo'
        elif any(keyword in title.lower() for keyword in ['desenvolvimento', 'saas', 'aplicativo', 'empresa individual']):
            course['category'] = 'Desenvolvimento de Aplicações'
        elif any(keyword in title.lower() for keyword in ['chatgpt', 'prompt', 'assistente inteligente']):
            course['category'] = 'Aplicações de Ferramentas de IA'
        elif any(keyword in title.lower() for keyword in ['marketing', 'marca', 'ip', 'e-commerce', 'copywriting']):
            course['category'] = 'Marketing Digital'
        elif any(keyword in title.lower() for keyword in ['escritório', 'produtividade']):
            course['category'] = 'Produtividade no Escritório'
        elif 'direito' in title.lower():
            course['category'] = 'Lei e Conformidade'
        elif 'educação' in title.lower():
            course['category'] = 'Aplicações Educacionais'
        else:
            course['category'] = 'Desenvolvimento Profissional'
        
        # Generate a URL-friendly slug
        slug = re.sub(r'[^\w\s]', '', title)
        slug = re.sub(r'\s+', '-', slug).lower()
        course['slug'] = slug
        
        # Add a teaser text
        teaser_words = description.split()[:15] if description else []
        course['teaser'] = ' '.join(teaser_words) + '...' if teaser_words else "Saiba mais..."
        
        # Adiciona flag de destaque para alguns cursos (para destaques na homepage)
        course['featured'] = (len(courses) % 3 == 0)
        
        # Adiciona nível de dificuldade (para filtro)
        if len(chapters) <= 5:
            course['difficulty'] = 'Iniciante'
        else:
            course['difficulty'] = 'Intermediário' if level == 'Básico' else 'Avançado'
        
        # Adiciona tempo estimado de conclusão
        course['completion_time'] = f"{len(chapters) * 3} dias"
        
        # Adiciona os tópicos do que você vai aprender
        course['what_youll_learn'] = [
            f"Domine as habilidades essenciais de {title.split('com')[-1].split('e')[0] if 'com' in title else title}",
            f"Do zero até ser capaz de concluir projetos relacionados a {course['category']}",
            f"Adquira experiência prática e insights do setor",
            f"Aprenda as mais recentes ferramentas e aplicações de IA"
        ]
        
        # Sample testimonials
        names = ["Li Ming", "Wang Hua", "Zhang Wei", "Zhao Min", "Chen Jing", "Lin Xiaoman", "Guo Zhiqiang", "Huang Jianguo"]
        testimonials = [
            "Este curso me ajudou a encontrar uma nova direção profissional, muito prático!",
            "Conteúdo claro e acessível, mesmo iniciantes podem aprender rapidamente.",
            "Instrutores experientes compartilharam muitas dicas práticas, recomendo!",
            "Curso bem estruturado, pude aplicar diretamente no meu trabalho após terminar.",
            "Excelente custo-benefício, aprendi muitas habilidades úteis."
        ]
        
        course['testimonials'] = [
            {
                'name': names[i % len(names)],
                'comment': testimonials[i % len(testimonials)],
                'avatar': f'/images/testimonials/user_{i+1}.jpg',
                'rating': 5 if i % 3 == 0 else 4
            } for i in range(2)  # 2 testimonials per course
        ]
        
        courses.append(course)
    
    return courses

# Extract courses from each section
basic_courses = extract_courses_from_section(basic_section, "Básico")
advanced_courses = extract_courses_from_section(advanced_section, "Avançado")

# Combine all courses
all_courses = []
for i, course in enumerate(basic_courses + advanced_courses):
    course['id'] = i + 1  # Reassign ids to ensure they're sequential
    all_courses.append(course)

# Try another approach if both methods failed to extract courses
if not all_courses:
    # Direct line-by-line parsing approach
    lines = content.split('\n')
    current_course = None
    current_level = "基础"  # Default
    courses = []
    
    for line in lines:
        if "【Cursos Básicos】" in line:
            current_level = "Básico"
            continue
        elif "【Cursos Avançados】" in line:
            current_level = "Avançado"
            continue
        
        # Verifica o título do curso
        title_match = re.search(r'\d+\|Título do Curso：(.*)', line)
        if title_match:
            # Salva o curso anterior se existir
            if current_course:
                courses.append(current_course)
            
            # Start new course
            current_course = {
                'id': len(courses) + 1,
                'title': title_match.group(1).strip(),
                'description': '',
                'chapters': [],
                'level': current_level
            }
            continue
        
        # Add description
        desc_match = re.search(r'\d+\|Descrição do Curso：(.*)', line)
        if desc_match and current_course:
            current_course['description'] = desc_match.group(1).strip()
            continue
        
        # Adiciona capítulo
        chapter_match = re.search(r'\d+\|- (Capítulo \d+\s+.*)', line)
        if chapter_match and current_course:
            current_course['chapters'].append(chapter_match.group(1).strip())

    # Add the last course
    if current_course:
        courses.append(current_course)
    
    # If we found courses this way, process them with additional fields
    if courses:
        all_courses = []
        for i, course in enumerate(courses):
            # Add instructor info
            course['instructor'] = {
                'name': 'Instrutor Especializado',
                'bio': 'Possui vasta experiência prática em aplicações de IA, focado em ajudar alunos a dominar o uso prático das ferramentas de IA',
                'avatar': f'/images/instructors/instructor_{(i % 5) + 1}.jpg'
            }
            
            # Add other fields
            course['cover_image'] = f'/images/courses/course_{i+1}.jpg'
            course['duration'] = f'{len(course["chapters"]) * 2}小时'
            course['students_enrolled'] = 100 + (i * 50)
            course['rating'] = round(4 + (i % 10) / 10, 1)
            
            # Add category based on title
            title = course['title']
            if any(keyword in title.lower() for keyword in ['vídeos curtos', 'imagem e texto', 'mangá', 'criação', 'música', 'roteiro']):
                course['category'] = 'Criação de Conteúdo'
            elif any(keyword in title.lower() for keyword in ['desenvolvimento', 'saas', 'aplicativo', 'empresa individual']):
                course['category'] = 'Desenvolvimento de Aplicações'
            elif any(keyword in title.lower() for keyword in ['chatgpt', 'prompt', 'assistente inteligente']):
                course['category'] = 'Aplicações de Ferramentas de IA'
            elif any(keyword in title.lower() for keyword in ['marketing', 'marca', 'ip', 'e-commerce', 'copywriting']):
                course['category'] = 'Marketing Digital'
            elif any(keyword in title.lower() for keyword in ['escritório', 'produtividade']):
                course['category'] = 'Produtividade no Escritório'
            elif 'direito' in title.lower():
                course['category'] = 'Lei e Conformidade'
            elif 'educação' in title.lower():
                course['category'] = 'Aplicações Educacionais'
            else:
                course['category'] = 'Desenvolvimento Profissional'
                
            # Generate slug
            slug = re.sub(r'[^\w\s]', '', title)
            slug = re.sub(r'\s+', '-', slug).lower()
            course['slug'] = slug
            
            # Add other MasterClass style fields
            course['teaser'] = course['description'][:50] + '...' if course['description'] else "Saiba mais..."
            course['featured'] = (i % 3 == 0)
            course['difficulty'] = 'Iniciante' if len(course['chapters']) <= 5 else ('Intermediário' if course['level'] == 'Básico' else 'Avançado')
            course['completion_time'] = f"{len(course['chapters']) * 3} dias"
            course['what_youll_learn'] = [
                f"Domine as habilidades essenciais de {title.split('com')[-1].split('e')[0] if 'com' in title else title}",
                f"Do zero até ser capaz de concluir projetos relacionados a {course['category']}",
                f"Adquira experiência prática e insights do setor",
                f"Aprenda as mais recentes ferramentas e aplicações de IA"
            ]
            
            # Adiciona depoimentos
            names = ["Li Ming", "Wang Hua", "Zhang Wei", "Zhao Min", "Chen Jing", "Lin Xiaoman", "Guo Zhiqiang", "Huang Jianguo"]
            testimonials = [
                "Este curso me ajudou a encontrar uma nova direção profissional, muito prático!",
                "Conteúdo claro e acessível, mesmo iniciantes podem aprender rapidamente.",
                "Instrutores experientes compartilharam muitas dicas práticas, recomendo!",
                "Curso bem estruturado, pude aplicar diretamente no meu trabalho após terminar.",
                "Excelente custo-benefício, aprendi muitas habilidades úteis."
            ]
            
            course['testimonials'] = [
                {
                    'name': names[j % len(names)],
                    'comment': testimonials[j % len(testimonials)],
                    'avatar': f'/images/testimonials/user_{j+1}.jpg',
                    'rating': 5 if j % 3 == 0 else 4
                } for j in range(2)  # 2 testimonials per course
            ]
            
            all_courses.append(course)

# Save the processed courses to JSON
with open('/data/chats/ejf70/workspace/processed_courses.json', 'w', encoding='utf-8') as out_file:
    json.dump(all_courses, out_file, ensure_ascii=False, indent=2)

print(f"Processou com sucesso {len(all_courses)} cursos e salvou em processed_courses.json")
if all_courses:
    print(f"Categorias dos cursos: {set(course['category'] for course in all_courses)}")
    print(f"Níveis dos cursos: {set(course['level'] for course in all_courses)}")

In [ ]:
import json
import re

# Read the course data file
with open('/data/chats/ejf70/workspace/uploads/curso.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# Initialize lists to store courses
courses = []
current_level = "Básico"  # Nível padrão
current_course = None

# Processa linha por linha
lines = content.split('\n')
for line in lines:
    # Verifica marcadores de nível
    if "【Cursos Básicos】" in line:
        current_level = "Básico"
        continue
    elif "【Cursos Avançados】" in line:
        current_level = "Avançado"
        continue

    # Check for course title
    title_match = re.search(r'\d+\|Título do Curso：(.*)', line)
    if title_match:
        # Salva o curso anterior, se existir
        if current_course:
            courses.append(current_course)
        
        # Inicia um novo curso
        current_course = {
            'id': len(courses) + 1,
            'title': title_match.group(1).strip(),
            'description': '',
            'chapters': [],
            'level': current_level
        }
        continue
    
    # Verifica a descrição do curso
    if current_course:  # Só prossegue se houver um curso atual
        desc_match = re.search(r'\d+\|Descrição do Curso：(.*)', line)
        if desc_match:
            current_course['description'] = desc_match.group(1).strip()
            continue
        
        # Check for chapters
        chapter_match = re.search(r'\d+\|- (Capítulo \d+.*)', line)
        if chapter_match:
            current_course['chapters'].append(chapter_match.group(1).strip())

# Adiciona o último curso, se existir
if current_course:
    courses.append(current_course)

# Melhora os dados do curso com campos adicionais no estilo MasterClass
for i, course in enumerate(courses):
    # Adiciona informações do instrutor
    course['instructor'] = {
        'name': f'Instrutor Especializado {i+1}',
        'bio': 'Possui vasta experiência prática em aplicações de IA, focado em ajudar alunos a dominar o uso prático das ferramentas de IA',
        'avatar': f'/images/instructors/instructor_{(i % 5) + 1}.jpg'
    }
    
    # Add course metadata
    course['cover_image'] = f'/images/courses/course_{i+1}.jpg'
    course['duration'] = f'{len(course["chapters"]) * 2}小时'
    course['students_enrolled'] = 100 + (i * 50)
    course['rating'] = round(4 + (i % 10) / 10, 1)
    
    # Add category based on title keywords
    title = course['title'].lower()
    if any(keyword in title for keyword in ['vídeos curtos', 'imagem e texto', 'mangá', 'criação', 'música', 'roteiro']):
        course['category'] = 'Criação de Conteúdo'
    elif any(keyword in title for keyword in ['desenvolvimento', 'saas', 'aplicativo', 'empresa individual']):
        course['category'] = 'Desenvolvimento de Aplicações'
    elif any(keyword in title for keyword in ['chatgpt', 'prompt', 'assistente inteligente']):
        course['category'] = 'Aplicações de Ferramentas de IA'
    elif any(keyword in title for keyword in ['marketing', 'marca', 'ip', 'e-commerce', 'copywriting']):
        course['category'] = 'Marketing Digital'
    elif any(keyword in title for keyword in ['escritório', 'produtividade']):
        course['category'] = 'Produtividade no Escritório'
    elif 'direito' in title:
        course['category'] = 'Lei e Conformidade'
    elif 'educação' in title:
        course['category'] = 'Aplicações Educacionais'
    else:
        course['category'] = 'Desenvolvimento Profissional'
    
    # Create URL-friendly slug
    slug = re.sub(r'[^\w\s]', '', course['title'])
    slug = re.sub(r'\s+', '-', slug).lower()
    course['slug'] = slug
    
    # Add teaser
    teaser_words = course['description'].split()[:15]
    course['teaser'] = ' '.join(teaser_words) + '...' if teaser_words else 'Saiba mais...'
    
    # Adiciona flag de destaque
    course['featured'] = (i % 3 == 0)
    
    # Adiciona nível de dificuldade
    if len(course['chapters']) <= 5:
        course['difficulty'] = 'Iniciante'
    else:
        course['difficulty'] = 'Intermediário' if course['level'] == 'Básico' else 'Avançado'
    
    # Adiciona estimativa de tempo para conclusão
    course['completion_time'] = f"{len(course['chapters']) * 3} dias"
    
    # Adiciona objetivos de aprendizagem
    course['what_youll_learn'] = [
        f"Domine as habilidades essenciais de {course['title'].split('com')[-1].split('e')[0] if 'com' in course['title'] else course['title']}",
        f"Do zero até ser capaz de concluir projetos relacionados a {course['category']}",
        f"Adquira experiência prática e insights do setor",
        f"Aprenda as mais recentes ferramentas e técnicas de aplicação de IA"
    ]
    
    # Add testimonials
    names = ["Li Ming", "Wang Hua", "Zhang Wei", "Zhao Min", "Chen Jing", "Lin Xiaoman", "Guo Zhiqiang", "Huang Jianguo"]
    testimonials = [
        "Este curso me ajudou a encontrar uma nova direção profissional, muito prático!",
        "Conteúdo claro e acessível, mesmo iniciantes podem aprender rapidamente.",
        "Instrutores experientes compartilharam muitas dicas práticas, recomendo!",
        "Curso bem estruturado, pude aplicar diretamente no meu trabalho após terminar.",
        "Excelente custo-benefício, aprendi muitas habilidades úteis."
    ]
    
    course['testimonials'] = [
        {
            'name': names[j % len(names)],
            'comment': testimonials[j % len(testimonials)],
            'avatar': f'/images/testimonials/user_{j+1}.jpg',
            'rating': 5 if j % 3 == 0 else 4
        } for j in range(2)  # 2 testimonials per course
    ]

# Save processed courses to JSON file
with open('/data/chats/ejf70/workspace/processed_courses.json', 'w', encoding='utf-8') as out_file:
    json.dump(courses, out_file, ensure_ascii=False, indent=2)

print(f"Processou com sucesso {len(courses)} cursos e salvou em processed_courses.json")
if courses:
    print(f"Categorias dos cursos: {set(course['category'] for course in courses)}")
    print(f"Níveis dos cursos: {set(course['level'] for course in courses)}")

In [ ]:
import json
import re

# Read the course data file
with open('/data/chats/ejf70/workspace/uploads/curso.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# Initialize variables for parsing
courses = []
current_level = "基础"  # Default starting level
current_course = None

# Parse the file content
lines = content.split('\n')
for line in lines:
    # Verifica os cabeçalhos de categoria
    if "【Cursos Básicos】" in line:
        current_level = "Básico"
        continue
    elif "【Cursos Avançados】" in line:
        current_level = "Avançado"
        continue
    
    # Extrai o título do curso - remove números de linha se presentes
    title_match = re.search(r'(?:\d+\|)?Título do Curso：(.*)', line)
    if title_match:
        # Save previous course if exists
        if current_course:
            courses.append(current_course)
        
        # Start new course
        current_course = {
            'id': len(courses) + 1,
            'title': title_match.group(1).strip(),
            'description': '',
            'chapters': [],
            'level': current_level
        }
        continue
    
    # Process only if we have a current course
    if current_course:
        # Extrai a descrição do curso - remove números de linha se presentes
        desc_match = re.search(r'(?:\d+\|)?Descrição do Curso：(.*)', line)
        if desc_match:
            current_course['description'] = desc_match.group(1).strip()
            continue
        
        # Extrai informações do capítulo - remove números de linha se presentes
        chapter_match = re.search(r'(?:\d+\|)?- (Capítulo \d+.*)', line)
        if chapter_match:
            current_course['chapters'].append(chapter_match.group(1).strip())

# Add the last course if there is one
if current_course:
    courses.append(current_course)

# Melhora os dados do curso com campos adicionais para estilo MasterClass
for i, course in enumerate(courses):
    # Adiciona informações do instrutor - usando nomes chineses realistas para instrutores
    nomes_instrutores = ["Li Mingyuan", "Wang Zhiqiang", "Zhang Tianqi", "Chen Siyuan", "Lin Yuting"]
    bios_instrutores = [
        "Especialista em aplicações de IA, 10 anos de experiência em produtos de internet, focado em pesquisa de IA + marca pessoal",
        "Ex-diretor técnico de grande empresa, praticante experiente em desenvolvimento de IA, especialista em simplificação técnica",
        "Criador de conteúdo sênior, especialista em aplicações criativas de IA, ajudou centenas de empresas na transformação por IA",
        "Especialista em operações de comércio eletrônico, proficiente em estratégias de marketing de IA, autor de 'Aplicações Comerciais de IA'",
        "Especialista em produtividade de escritório, autoridade em integração de ferramentas de IA, instrutor de treinamento corporativo de IA"
    ]
    
    course['instructor'] = {
        'name': nomes_instrutores[i % len(nomes_instrutores)],
        'bio': bios_instrutores[i % len(bios_instrutores)],
        'avatar': f'/images/instructors/instructor_{(i % 5) + 1}.jpg'
    }
    # Add course metadata
    course['cover_image'] = f'/images/courses/course_{i+1}.jpg'
    course['duration'] = f'{len(course["chapters"]) * 2}小时'
    course['students_enrolled'] = 100 + (i * 50)
    course['rating'] = round(4.5 + (i % 10) / 20, 1)  # Ratings between 4.5-5.0
    
    # Add category based on title keywords - more precise categorization
    title = course['title'].lower()
    if any(keyword in title for keyword in ['vídeos curtos', 'imagem e texto', 'mangá', 'criação', 'música', 'roteiro']):
        course['category'] = 'Criação de Conteúdo'
    elif any(keyword in title for keyword in ['desenvolvimento', 'saas', 'aplicativo', 'empresa individual']):
        course['category'] = 'Desenvolvimento de Aplicações'
    elif any(keyword in title for keyword in ['chatgpt', 'prompt', 'assistente inteligente']):
        course['category'] = 'Aplicações de Ferramentas de IA'
    elif any(keyword in title for keyword in ['marketing', 'marca', 'ip', 'e-commerce', 'copywriting']):
        course['category'] = 'Marketing Digital'
    elif any(keyword in title for keyword in ['escritório', 'produtividade']):
        course['category'] = 'Produtividade no Escritório'
    elif 'direito' in title:
        course['category'] = 'Lei e Conformidade'
    elif 'educação' in title:
        course['category'] = 'Aplicações Educacionais'
    else:
        course['category'] = 'Desenvolvimento Profissional'
    
    # Create URL-friendly slug
    slug = re.sub(r'[^\w\s]', '', course['title'])
    slug = re.sub(r'\s+', '-', slug).lower()
    course['slug'] = slug
    
    # Add teaser - more engaging preview
    teaser_words = course['description'].split()[:12]
    course['teaser'] = ' '.join(teaser_words) + '...' if teaser_words else 'Explore como a IA pode transformar sua carreira...'
    
    # Adiciona flag de destaque - distribui os cursos destacados de forma mais equilibrada
    course['featured'] = (i % 4 == 0 or i % 7 == 0)  # Distribuição mais interessante
    
    # Adiciona nível de dificuldade - baseado no conteúdo e nível do curso
    chapter_count = len(course['chapters'])
    if chapter_count <= 5:
        course['difficulty'] = 'Nível Iniciante'
    else:
        course['difficulty'] = 'Nível Avançado' if course['level'] == 'Avançado' else 'Nível Básico'
    
    # Add completion time estimate - more realistic estimates
    course['completion_time'] = f"{chapter_count * 2 + 5} dias"
    
    # Adiciona objetivos de aprendizagem - mais específicos para cada curso
    title_focus = course['title'].split('com')[-1].split('e')[0] if 'com' in course['title'] else course['title']
    course['what_youll_learn'] = [
        f"Domine as habilidades e processos essenciais de {title_focus}",
        f"Do zero até ser capaz de concluir projetos práticos relacionados a {course['category']}",
        f"Adquira insights do setor e experiência prática",
        f"Construa sua própria metodologia de trabalho com IA"
    ]
    
    # Adiciona depoimentos - mais personalizados
    names = ["Li Xiaohua", "Wang Jianming", "Zhang Siqi", "Zhao Yunfeng", "Chen Mingliang", "Lin Xiaoman", "Guo Zhiqiang", "Huang Jianguo"]
    jobs = ["Gerente de Produto", "Freelancer", "Criador de Conteúdo", "Marketing", "Engenheiro de Desenvolvimento", "Empresário", "Professor Universitário", "Designer"]
    
    testimonial_templates = [
        "Este curso me ajudou a {outcome}, {benefit}!",
        "Em {timeframe} dominei {skill}, {result}.",
        "O instrutor é {quality}, {learning}, {recommendation}.",
        "O curso é {feature}, {application}.",
        "Excelente custo-benefício! {before}, agora {after}."
    ]
    
    outcomes = [
        "encontrar uma nova direção profissional",
        "aumentar a eficiência em 3 vezes",
        "resolver um problema que me incomodava há muito tempo",
        "adquirir habilidades práticas",
        "abrir uma nova visão para aplicações de IA"
    ]
    
    benefits = [
        "muito prático",
        "vale cada centavo",
        "altamente recomendado para colegas",
        "aplicável na prática",
        "retorno sobre investimento extremamente alto"
    ]
    
    course['testimonials'] = []
    for j in range(2):  # 2 depoimentos por curso
        idx = (i + j) % len(names)
        testimonial = {
            'name': names[idx],
            'job': jobs[idx],
            'comment': f"Este curso {course['title']} é excelente! {outcomes[idx % len(outcomes)]}, {benefits[j % len(benefits)]}.",
            'avatar': f'/images/testimonials/user_{idx+1}.jpg',
            'rating': 5 if idx % 3 == 0 else 4
        }
        course['testimonials'].append(testimonial)
        
    # Adiciona informações de preços para estilo MasterClass
    course['pricing'] = {
        'original': 399 + (i % 5) * 100,
        'current': 299 + (i % 5) * 100,
        'discount_percent': 25 if i % 3 == 0 else 0
    }
    
    # Add trailer video (placeholder)
    course['trailer_video'] = f'/videos/courses/course_{i+1}_trailer.mp4'
    
    # Add related skills tags
    skill_tags = {
        'Criação de Conteúdo': ['Escrita Criativa', 'Conteúdo Gerado por IA', 'Design Visual', 'Produção de Vídeos Curtos', 'Planejamento de Conteúdo'],
        'Desenvolvimento de Aplicações': ['Desenvolvimento Sem Código', 'Integração de API', 'Experiência do Usuário', 'Design de Produto', 'Automação'],
        'Aplicações de Ferramentas de IA': ['Engenharia de Prompt', 'Modelos de Linguagem Grande', 'Construção de Base de Conhecimento', 'Fluxo de Trabalho com IA', 'Scripts Automatizados'],
        'Marketing Digital': ['Marketing de Conteúdo', 'Mídias Sociais', 'Criatividade em Anúncios', 'Crescimento de Usuários', 'Análise de Dados'],
        'Produtividade no Escritório': ['Otimização de Fluxo de Trabalho', 'Relatórios Automatizados', 'Colaboração em Equipe', 'Gerenciamento de Projetos', 'Assistente de IA'],
        'Lei e Conformidade': ['Conformidade de Dados', 'Propriedade Intelectual', 'Ética em IA', 'Riscos Legais', 'Proteção de Privacidade'],
        'Aplicações Educacionais': ['Aprendizagem Personalizada', 'Design Instrucional', 'Avaliação de Aprendizagem', 'Desenvolvimento de Conteúdo', 'Experiência Interativa'],
        'Desenvolvimento Profissional': ['Aprimoramento de Habilidades', 'Marca Pessoal', 'Freelancing', 'Desenvolvimento de Renda Extra', 'Planejamento de Carreira']
    }
    
    course['skill_tags'] = skill_tags.get(course['category'], [])[:3]

# Salva os cursos processados no arquivo JSON
with open('/data/chats/ejf70/workspace/processed_courses.json', 'w', encoding='utf-8') as out_file:
    json.dump(courses, out_file, ensure_ascii=False, indent=2)

print(f"Processou com sucesso {len(courses)} cursos e salvou em processed_courses.json")
if courses:
    print(f"Categorias dos cursos: {set(course['category'] for course in courses)}")
    print(f"Níveis dos cursos: {set(course['level'] for course in courses)}")
    print("Exemplo de curso:")
    print(json.dumps(courses[0], ensure_ascii=False, indent=2))

In [ ]:
I'll create a MasterClass-style website homepage for the online course subscription platform. The design will feature a dark elegant theme with high-quality visuals and clean typography.

```python
import json
from pathlib import Path

# Read the processed course data
with open('/data/chats/ejf70/workspace/processed_courses.json', 'r', encoding='utf-8') as file:
    courses = json.load(file)

# Create directories for the website if they don't exist
Path('/data/chats/ejf70/workspace/website').mkdir(exist_ok=True)
Path('/data/chats/ejf70/workspace/website/css').mkdir(exist_ok=True)
Path('/data/chats/ejf70/workspace/website/js').mkdir(exist_ok=True)
Path('/data/chats/ejf70/workspace/website/images').mkdir(exist_ok=True)

# Sort and organize courses
featured_courses = [course for course in courses if course.get('featured', False)]
basic_courses = [course for course in courses if course['level'] == '基础']
advanced_courses = [course for course in courses if course['level'] == '进阶']

# Helper function to get courses by category
def get_courses_by_category(category, limit=4):
    return [c for c in courses if c.get('category') == category][:limit]

# Create HTML file
html_content = """
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Academia - Plataforma de Assinatura de Cursos Online de IA</title>
    <link rel="stylesheet" href="css/styles.css">
    <link href="https://fonts.googleapis.com/css2?family=Noto+Sans+SC:wght@300;400;500;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
</head>
<body>
    <div class="preloader">
        <div class="preloader-content">
            <div class="logo">AI<span>学堂</span></div>
            <div class="spinner"></div>
        </div>
    </div>

    <header class="main-header">
        <div class="header-container">
            <div class="logo">AI<span>Academia</span></div>
            <nav class="main-nav">
                <ul class="nav-links">
                    <li><a href="#featured" class="nav-link">Cursos em Destaque</a></li>
                    <li><a href="#categories" class="nav-link">Categorias de Cursos</a></li>
                    <li><a href="#instructors" class="nav-link">Equipe de Instrutores</a></li>
                    <li><a href="#testimonials" class="nav-link">Avaliações dos Alunos</a></li>
                </ul>
            </nav>
            <div class="header-actions">
                <div class="search-container">
                    <input type="text" placeholder="Pesquisar cursos..." class="search-input">
                    <button class="search-btn"><i class="fas fa-search"></i></button>
                </div>
                <a href="#" class="btn btn-outline">Entrar</a>
                <a href="#" class="btn btn-primary">Começar a aprender</a>
            </div>
            <button class="mobile-menu-btn">
                <span></span><span></span><span></span>
            </button>
        </div>
    </header>

    <main>
        <!-- Hero Section -->
        <section class="hero-section">
            <div class="hero-content">
                <h1 class="hero-title">Use IA para impulsionar<br> sua competitividade profissional</h1>
                <p class="hero-subtitle">Do uso básico a habilidades avançadas, descubra como a IA pode transformar seu trabalho e vida</p>
                <div class="hero-cta">
                    <a href="#featured" class="btn btn-primary btn-large">Explorar cursos</a>
                    <a href="#" class="btn btn-outline btn-large">Conheça a assinatura</a>
                </div>
                <div class="hero-stats">
                    <div class="stat-item">
                        <span class="stat-number">17+</span>
                        <span class="stat-label">Cursos Premium</span>
                    </div>
                    <div class="stat-item">
                        <span class="stat-number">7</span>
                        <span class="stat-label">Áreas de Especialização</span>
                    </div>
                    <div class="stat-item">
                        <span class="stat-number">5000+</span>
                        <span class="stat-label">Alunos</span>
                    </div>
                </div>
            </div>
            <div class="hero-image-container">
                <div class="hero-image"></div>
                <div class="floating-card card-1">
                    <div class="card-icon"><i class="fas fa-brain"></i></div>
                    <div class="card-text">Treinamento em Pensamento de IA</div>
                </div>
                <div class="floating-card card-2">
                    <div class="card-icon"><i class="fas fa-rocket"></i></div>
                    <div class="card-text">Aumento de eficiência em 10x</div>
                </div>
            </div>
        </section>

        <!-- Seção de Cursos em Destaque -->
        <section id="featured" class="featured-section section">
            <div class="section-header">
                <h2 class="section-title">Cursos em Destaque</h2>
                <p class="section-subtitle">Cursos populares de aplicação de IA, cuidadosamente elaborados por instrutores profissionais</p>
            </div>
            <div class="courses-grid">
"""

# Add featured courses
for course in featured_courses[:4]:  # Limit to 4 featured courses
    html_content += f"""
                <div class="course-card featured">
                    <div class="course-image-container">
                        <div class="course-image" style="background-image: url('{course['cover_image']}');"></div>
                        <div class="course-level">{course['level']}</div>
                        <div class="course-duration"><i class="far fa-clock"></i> {course['duration']}</div>
                    </div>
                    <div class="course-content">
                        <div class="course-category">{course['category']}</div>
                        <h3 class="course-title">{course['title']}</h3>
                        <p class="course-description">{course['teaser']}</p>
                        <div class="course-meta">
                            <div class="course-instructor">
                                <img src="{course['instructor']['avatar']}" alt="{course['instructor']['name']}" class="instructor-avatar">
                                <span>{course['instructor']['name']}</span>
                            </div>
                            <div class="course-rating">
                                <span class="rating-score">{course['rating']}</span>
                                <div class="rating-stars">
                                    {"".join(['<i class="fas fa-star"></i>' for _ in range(int(course['rating']))])}
                                    {"<i class='fas fa-star-half-alt'></i>" if course['rating'] % 1 >= 0.5 else ""}
                                </div>
                            </div>
                        </div>
                    </div>
                    <a href="#" class="course-link" data-id="{course['id']}">查看详情</a>
                </div>
    """

html_content += """
            </div>
        </section>

        <!-- Categories Section -->
        <section id="categories" class="categories-section section">
            <div class="section-header">
                <h2 class="section-title">Categorias de Cursos</h2>
                <p class="section-subtitle">Explore os cursos conforme seus objetivos de aprendizado e interesses</p>
            </div>
            <div class="categories-tabs">
                <div class="tabs-header">
                    <button class="tab-btn active" data-tab="ai-tools">Aplicações de Ferramentas de IA</button>
                    <button class="tab-btn" data-tab="content">Criação de Conteúdo</button>
                    <button class="tab-btn" data-tab="development">Desenvolvimento de Aplicações</button>
                    <button class="tab-btn" data-tab="marketing">Marketing Digital</button>
                    <button class="tab-btn" data-tab="efficiency">Produtividade no Escritório</button>
                </div>
                
                <div class="tabs-content">
"""

# AI Tools Category Tab
ai_tools_courses = get_courses_by_category("Aplicações de Ferramentas de IA")
html_content += """
                    <div class="tab-pane active" id="ai-tools">
                        <div class="category-info">
                            <div class="category-description">
                                <h3>Aplicações de Ferramentas de IA</h3>
                                <p>Domine as habilidades práticas das ferramentas avançadas de IA como ChatGPT e Midjourney, sem necessidade de programação, tornando a IA sua assistente poderosa.</p>
                                <ul class="category-features">
                                    <li><i class="fas fa-check"></i> Engenharia de prompt e aplicações avançadas</li>
                                    <li><i class="fas fa-check"></i> Automação de fluxos de trabalho</li>
                                    <li><i class="fas fa-check"></i> Construção de assistentes personalizados de IA</li>
                                </ul>
                                <a href="#" class="btn btn-outline">Ver todos</a>
                            </div>
                            <div class="category-courses">
"""

for course in ai_tools_courses:
    html_content += f"""
                                <div class="category-course-card">
                                    <div class="category-course-image" style="background-image: url('{course['cover_image']}');"></div>
                                    <div class="category-course-content">
                                        <h4>{course['title']}</h4>
                                        <div class="category-course-meta">
                                            <span>{course['duration']}</span>
                                            <span>{course['level']}</span>
                                        </div>
                                    </div>
                                </div>
    """

html_content += """
                            </div>
                        </div>
                    </div>
"""

# Content Creation Category Tab
content_courses = get_courses_by_category("Criação de Conteúdo")
html_content += """
                    <div class="tab-pane" id="content">
                        <div class="category-info">
                            <div class="category-description">
                                <h3>Criação de Conteúdo</h3>
                                <p>Domine as ferramentas de geração de conteúdo com IA para criar facilmente vídeos curtos, imagens com texto, áudio e outros formatos criativos, aumentando a eficiência da produção.</p>
                                <ul class="category-features">
                                    <li><i class="fas fa-check"></i> Criação de imagens e vídeos com IA</li>
                                    <li><i class="fas fa-check"></i> Distribuição automatizada de conteúdo</li>
                                    <li><i class="fas fa-check"></i> Construção de IP criativo</li>
                                </ul>
                                <a href="#" class="btn btn-outline">Ver todos</a>
                            </div>
                            <div class="category-courses">
"""

for course in content_courses:
    html_content += f"""
                                <div class="category-course-card">
                                    <div class="category-course-image" style="background-image: url('{course['cover_image']}');"></div>
                                    <div class="category-course-content">
                                        <h4>{course['title']}</h4>
                                        <div class="category-course-meta">
                                            <span>{course['duration']}</span>
                                            <span>{course['level']}</span>
                                        </div>
                                    </div>
                                </div>
    """

html_content += """
                            </div>
                        </div>
                    </div>
"""

# Add other category tabs placeholders
html_content += """
                    <div class="tab-pane" id="development">
                        <!-- Development courses tab content will be populated by JavaScript -->
                    </div>
                    <div class="tab-pane" id="marketing">
                        <!-- Marketing courses tab content will be populated by JavaScript -->
                    </div>
                    <div class="tab-pane" id="efficiency">
                        <!-- Efficiency courses tab content will be populated by JavaScript -->
                    </div>
                </div>
            </div>
        </section>

        <!-- Learning Path Section -->
        <section class="learning-path-section section">
            <div class="section-header">
                <h2 class="section-title">Caminho de Aprendizagem</h2>
                <p class="section-subtitle">Do básico ao uso profissional, desenhamos um caminho claro de aprendizagem</p>
            </div>
            <div class="learning-path-container">
                <div class="path-navigator">
                    <div class="path-node active">
                        <div class="node-number">01</div>
                        <div class="node-content">
                            <h3>Introdução Básica</h3>
                            <p>Compreenda conceitos básicos e ferramentas de IA</p>
                        </div>
                    </div>
                    <div class="path-node">
                        <div class="node-number">02</div>
                        <div class="node-content">
                            <h3>Aplicações Práticas</h3>
                            <p>Domine o uso da IA em cenários específicos</p>
                        </div>
                    </div>
                    <div class="path-node">
                        <div class="node-number">03</div>
                        <div class="node-content">
                            <h3>Habilidades Avançadas</h3>
                            <p>Aprenda profundamente técnicas e métodos avançados</p>
                        </div>
                    </div>
                    <div class="path-node">
                        <div class="node-number">04</div>
                        <div class="node-content">
                            <h3>Prática Profissional</h3>
                            <p>Projeto real prático e entrega de resultados</p>
                        </div>
                    </div>
                </div>
                <div class="path-content">
                    <div class="path-detail active">
                        <h3>Introdução Básica à IA</h3>
                        <p>Ideal para iniciantes, compreenda os conceitos básicos, princípios de funcionamento e cenários comuns de aplicação de ferramentas de IA, estabelecendo uma base de pensamento em IA.</p>
                        <div class="path-courses">
"""

# Add basic level courses to the first path node
for course in basic_courses[:3]:
    html_content += f"""
                            <div class="path-course">
                                <div class="path-course-image" style="background-image: url('{course['cover_image']}');"></div>
                                <div class="path-course-info">
                                    <h4>{course['title']}</h4>
                                    <div class="path-course-meta">
                                        <span>{course['duration']}</span>
                                        <span class="difficulty">{course['difficulty']}</span>
                                    </div>
                                </div>
                            </div>
    """

html_content += """
                        </div>
                        <a href="#" class="btn btn-primary">Começar a aprender</a>
                    </div>
                    <!-- Outros detalhes do caminho serão exibidos/ocultados via JavaScript -->
                </div>
            </div>
        </section>

        <!-- Seção de Instrutores -->
        <section id="instructors" class="instructors-section section">
            <div class="section-header">
                <h2 class="section-title">Equipe de Instrutores de Elite</h2>
                <p class="section-subtitle">Instrutores profissionais da linha de frente da indústria, guiando você para dominar habilidades práticas</p>
            </div>
            <div class="instructors-container">
"""

# Get unique instructors
unique_instructors = {}
for course in courses:
    instructor = course['instructor']
    if instructor['name'] not in unique_instructors:
        unique_instructors[instructor['name']] = instructor

# Add instructors
for name, instructor in list(unique_instructors.items())[:5]:  # Limit to 5 instructors
    html_content += f"""
                <div class="instructor-card">
                    <div class="instructor-image" style="background-image: url('{instructor['avatar']}');"></div>
                    <h3 class="instructor-name">{instructor['name']}</h3>
                    <p class="instructor-title">{instructor['bio'].split('，')[0]}</p>
                    <p class="instructor-bio">{instructor['bio']}</p>
                    <div class="instructor-stats">
                        <div class="stat">
                            <span class="stat-number">{courses.count(instructor['name'])}</span>
                            <span class="stat-label">Cursos</span>
                        </div>
                        <div class="stat">
                            <span class="stat-number">4.8</span>
                            <span class="stat-label">Avaliação</span>
                        </div>
                        <div class="stat">
                            <span class="stat-number">1000+</span>
                            <span class="stat-label">Alunos</span>
                        </div>
                    </div>
                </div>
    """

html_content += """
            </div>
        </section>

        <!-- Seção de Depoimentos -->
        <section id="testimonials" class="testimonials-section section">
            <div class="section-header">
                <h2 class="section-title">Feedback dos Alunos</h2>
                <p class="section-subtitle">Veja como outros alunos aprimoraram suas habilidades através dos nossos cursos</p>
            </div>
            <div class="testimonials-slider">
"""

# Collect testimonials from different courses
all_testimonials = []
for course in courses:
    if 'testimonials' in course:
        for testimonial in course['testimonials']:
            testimonial['course_title'] = course['title']
            all_testimonials.append(testimonial)

# Add testimonials
for testimonial in all_testimonials[:6]:  # Limit to 6 testimonials
    html_content += f"""
                <div class="testimonial-card">
                    <div class="testimonial-header">
                        <div class="testimonial-avatar" style="background-image: url('{testimonial['avatar']}');"></div>
                        <div class="testimonial-author">
                            <h4 class="author-name">{testimonial['name']}</h4>
                            <p class="author-title">{testimonial.get('job', 'Aluno')}</p>
                        </div>
                        <div class="testimonial-rating">
                            {"".join(['<i class="fas fa-star"></i>' for _ in range(testimonial['rating'])])}
                        </div>
                    </div>
                    <div class="testimonial-content">
                        <p>"{testimonial['comment']}"</p>
                        <div class="testimonial-course">— Estudou {testimonial['course_title']}</div>
                    </div>
                </div>
    """

html_content += """
            </div>
        </section>

        <!-- Seção de Assinatura -->
        <section class="subscription-section">
            <div class="subscription-content">
                <h2>Seja membro, desbloqueie todos os cursos</h2>
                <p>Uma única assinatura para acessar todos os cursos e futuras atualizações, aprenda a qualquer hora, assista ilimitadamente</p>
                <div class="subscription-plans">
                    <div class="plan-card">
                        <div class="plan-header">
                            <h3>Assinatura Mensal</h3>
                            <div class="plan-price">
                                <span class="currency">¥</span>
                                <span class="amount">99</span>
                                <span class="period">/mês</span>
                            </div>
                        </div>
                        <ul class="plan-features">
                            <li><i class="fas fa-check"></i> Acesso ilimitado a todos os cursos</li>
                            <li><i class="fas fa-check"></i> Cancelamento a qualquer momento</li>
                            <li><i class="fas fa-check"></i> Suporte para aprendizado em múltiplos dispositivos</li>
                            <li><i class="fas fa-check"></i> Acesso à comunidade de aprendizado</li>
                        </ul>
                        <a href="#" class="btn btn-outline btn-block">Escolher este plano</a>
                    </div>
                    <div class="plan-card featured-plan">
                        <div class="plan-badge">Mais Popular</div>
                        <div class="plan-header">
                            <h3>Assinatura Anual</h3>
                            <div class="plan-price">
                                <span class="currency">¥</span>
                                <span class="amount">799</span>
                                <span class="period">/ano</span>
                            </div>
                            <div class="plan-save">Economize 33%</div>
                        </div>
                        <ul class="plan-features">
                            <li><i class="fas fa-check"></i> Acesso ilimitado a todos os cursos</li>
                            <li><i class="fas fa-check"></i> Inclui todos os novos cursos</li>
                            <li><i class="fas fa-check"></i> Suporte para aprendizado em múltiplos dispositivos</li>
                            <li><i class="fas fa-check"></i> Comunidade VIP exclusiva</li>
                            <li><i class="fas fa-check"></i> Sessões mensais de perguntas e respostas ao vivo</li>
                        </ul>
                        <a href="#" class="btn btn-primary btn-block">Escolher este plano</a>
                    </div>
                </div>
                <div class="subscription-guarantee">
                    <i class="fas fa-shield-alt"></i>
                    <p>Garantia de reembolso de 7 dias sem motivo, experimente sem riscos</p>
                </div>
            </div>
        </section>

        <!-- Seção CTA -->
        <section class="cta-section">
            <div class="cta-content">
                <h2>Pronto para aprimorar suas habilidades em IA?</h2>
                <p>Junte-se a mais de 5000 alunos e use ferramentas de IA para aumentar sua eficiência, criatividade e competitividade profissional</p>
                <a href="#" class="btn btn-primary btn-large">Comece agora</a>
            </div>
        </section>
    </main>

    <footer class="main-footer">
        <div class="footer-content">
            <div class="footer-logo">
                <div class="logo">AI<span>Academia</span></div>
                <p>Tornando o aprendizado de habilidades em IA simples e acessível</p>
            </div>
            <div class="footer-links">
                <div class="footer-column">
                    <h3>Cursos</h3>
                    <ul>
                        <li><a href="#">Todos os cursos</a></li>
                        <li><a href="#">Início básico</a></li>
                        <li><a href="#">Aprimoramento avançado</a></li>
                        <li><a href="#">Caminhos de aprendizagem</a></li>
                    </ul>
                </div>
                <div class="footer-column">
                    <h3>Sobre nós</h3>
                    <ul>
                        <li><a href="#">Equipe</a></li>
                        <li><a href="#">Recrutamento de instrutores</a></li>
                        <li><a href="#">Consultoria de parcerias</a></li>
                        <li><a href="#">Perguntas frequentes</a></li>
                    </ul>
                </div>
                <div class="footer-column">
                    <h3>Recursos</h3>
                    <ul>
                        <li><a href="#">Comunidade de aprendizado</a></li>
                        <li><a href="#">Biblioteca de ferramentas AI</a></li>
                        <li><a href="#">Tutoriais gratuitos</a></li>
                        <li><a href="#">Trabalhos dos alunos</a></li>
                    </ul>
                </div>
            </div>
            <div class="footer-newsletter">
                <h3>Assine para receber novidades</h3>
                <p>Receba as últimas atualizações de cursos, eventos e novidades em tecnologia AI</p>
                <form class="newsletter-form">
                    <input type="email" placeholder="Seu endereço de e-mail" required>
                    <button type="submit" class="btn btn-primary">Assinar</button>
                </form>
                <div class="social-links">
                    <a href="#"><i class="fab fa-weixin"></i></a>
                    <a href="#"><i class="fab fa-weibo"></i></a>
                    <a href="#"><i class="fab fa-zhihu"></i></a>
                    <a href="#"><i class="fab fa-bilibili"></i></a>
                </div>
            </div>
        </div>
        <div class="footer-bottom">
            <div class="copyright">© 2023 AI Academia. Todos os direitos reservados</div>
            <div class="footer-legal">
                <a href="#">Política de Privacidade</a>
                <a href="#">Termos de Serviço</a>
                <a href="#">Política de Reembolso</a>
            </div>
        </div>
    </footer>

    <script src="js/script.js"></script>
</body>
</html>
"""

# Create CSS file
css_content = """
/* Variables */
:root {
    /* Colors */
    --color-black: #0a0a0a;
    --color-dark: #121212;
    --color-gray-900: #1a1a1a;
    --color-gray-800: #2a2a2a;
    --color-gray-700: #333333;
    --color-gray-600: #666666;
    --color-gray-500: #888888;
    --color-gray-400: #999999;
    --color-gray-300: #cccccc;
    --color-gray-200: #dddddd;
    --color-gray-100: #eeeeee;
    --color-white: #ffffff;
    
    --color-primary: #ff4d4d;
    --color-primary-dark: #cc3333;
    --color-primary-light: #ff6666;
    
    --color-secondary: #333333;
    --color-accent: #8a2be2;
    
    /* Typography */
    --font-primary: 'Noto Sans SC', -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;
    
    /* Spacing */
    --space-xs: 0.25rem;
    --space-s: 0.5rem;
    --space-m: 1rem;
    --space-l: 1.5rem;
    --space-xl: 2rem;
    --space-xxl: 3rem;
    
    /* Border Radius */
    --radius-s: 4px;
    --radius-m: 8px;
    --radius-l: 12px;
    --radius-xl: 20px;
    
    /* Shadows */
    --shadow-sm: 0 2px 8px rgba(0, 0, 0, 0.15);
    --shadow-md: 0 4px 16px rgba(0, 0, 0, 0.1);
    --shadow-lg: 0 8px 30px rgba(0, 0, 0, 0.15);
    --shadow-inner: inset 0 2px 6px rgba(0, 0, 0, 0.15);
    
    /* Transitions */
    --transition-fast: 0.2s ease;
    --transition-medium: 0.3s ease;
    --transition-slow: 0.5s ease;
    
    /* Z-index */
    --z-index-dropdown: 100;
    --z-index-sticky: 200;
    --z-index-fixed: 300;
    --z-index-modal-backdrop: 400;
    --z-index-modal: 500;
    --z-index-popover: 600;
    --z-index-tooltip: 700;
    --z-index-preloader: 9999;
}

/* Base Styles */
* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

html {
    font-size: 16px;
    scroll-behavior: smooth;
}

body {
    font-family: var(--font-primary);
    background-color: var(--color-black);
    color: var(--color-white);
    line-height: 1.6;
    overflow-x: hidden;
    -webkit-font-smoothing: antialiased;
    -moz-osx-font-smoothing: grayscale;
}

h1, h2, h3, h4, h5, h6 {
    margin: 0 0 1rem;
    line-height: 1.3;
    font-weight: 700;
}

p {
    margin: 0 0 1.5rem;
}

a {
    color: var(--color-white);
    text-decoration: none;
    transition: color var(--transition-fast);
}

a:hover {
    color: var(--color-primary);
}

ul, ol {
    list-style: none;
}

img {
    max-width: 100%;
    height: auto;
}

button {
    cursor: pointer;
    font-family: var(--font-primary);
}

section {
    padding: 5rem 0;
}

.section {
    padding: 6rem 0;
}

.section-header {
    text-align: center;
    max-width: 800px;
    margin: 0 auto 4rem;
}

.section-title {
    font-size: 2.5rem;
    margin-bottom: 1rem;
}

.section-subtitle {
    color: var(--color-gray-400);
    font-size: 1.2rem;
}

.btn {
    display: inline-flex;
    align-items: center;
    justify-content: center;
    padding: 0.75rem 1.5rem;
    border-radius: var(--radius-m);
    font-weight: 500;
    text-align: center;
    cursor: pointer;
    transition: all var(--transition-fast);
    border: none;
}

.btn-primary {
    background: var(--color-primary);
    color: var(--color-white);
}

.btn-primary:hover {
    background: var(--color-primary-dark);
    color: var(--color-white);
    transform: translateY(-2px);
}

.btn-outline {
    background: transparent;
    color: var(--color-white);
    border: 2px solid var(--color-gray-600);
}

.btn

SyntaxError: unterminated string literal (detected at line 1) (3608273210.py, line 1)